# Technical Indicators Chart Visualizer

Interactive chart with technical indicators for ADAUSDT trading data visualization.

**13 Technical Indicators:**
- 📊 **Price**: Candlestick chart with OHLC data
- 📈 **Moving Averages**: EMA 20, 50, 200  
- ? **VWAP**: Volume Weighted Average Price
- 🎯 **Bollinger Bands**: Upper/lower bands with fill
- ⚡ **RSI**: With overbought/oversold levels
- 📈 **MACD**: Signal line and histogram
- 📊 **Volume**: Color-coded bars with MA 20

In [ ]:
# =============================================================================
# SETUP & CONFIGURATION
# =============================================================================

# Setup - Add src to Python path
import sys, os

project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import Libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings
warnings.filterwarnings('ignore')

# Project modules
from src.training.data_loader import DataLoader
from src.core.trading_types import ChartInterval

# Set plotly template
pio.templates.default = "plotly_dark"

# Symbol and timeframe settings
symbol = 'BTCUSDT'
time_frame:ChartInterval = '15m'

# Define which indicators to show on the chart by timeframe
CHART_FEATURES_CONFIG = {
    'main_chart': {
        'ema9': ['15m'],
        'ema21': ['15m'],
        'ema50': ['15m'],
        #'ema200': ['15m', '1h'],          # EMA200 only exists for 15m & 1h
        'vwap': ['15m'],                   # Only on 15m in final features
        #'bb_upper': ['15m'],               # Only on 15m
        #'bb_lower': ['15m']                # Only on 15m
    },
    'rsi_subplot': {
        'rsi': ['15m']
    },
    'macd_subplot': {
        #'macd': ['15m', '1h', 'D'],
        #'macd_hist': ['15m', '1h', 'D']
    },
    'volume_subplot': {
        'volume': ['15m'],
        'volume_ma20': ['15m']             # Only 15m
    }
}


# Indicator styling configuration
INDICATOR_STYLES = {
    # Bollinger Bands
    'bb_upper': {'width': 1, 'opacity': 0.6, 'dash': 'auto'},
    'bb_lower': {'width': 1, 'opacity': 0.6, 'dash': 'auto', 'fill': 'tonexty', 'fill_opacity': 0.1},
    
    # VWAP
    'vwap': {'width': 2, 'opacity': 0.9, 'dash': 'dot'},
    
    # Moving Averages (default styles)
    'ema5': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    'ema9': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    'ema13': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    'ema20': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    'ema21': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    'ema50': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    'ema200': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'},
    
    # Default for any unlisted indicators
    'default': {'width': 'auto', 'opacity': 0.8, 'dash': 'auto'}
}

# Color scheme for different timeframes
TIMEFRAME_COLORS = {
    '15m': '#ffd700',    # Gold
    '1h': '#ff6600',     # Orange  
    'D': '#ff0066',      # Pink
    'W': '#9370db',      # Purple
    'M': '#00ffff'       # Cyan
}

def get_available_features_by_timeframe(df, base_indicator, timeframes):
    """Detect available timeframe-specific features in the dataframe."""
    available = {}
    
    for tf in timeframes:
        if tf == '15m':
            # Primary timeframe usually has no suffix
            col_name = base_indicator
        else:
            # Other timeframes have suffix: indicator_timeframe
            col_name = f"{base_indicator}_{tf}"
        
        if col_name in df.columns:
            available[tf] = col_name
        else:
            print(f"  Missing: {col_name}")
    
    return available

def get_indicator_style(indicator, timeframe):
    """Get styling for an indicator based on configuration."""
    style = INDICATOR_STYLES.get(indicator, INDICATOR_STYLES['default']).copy()
    
    # Handle auto values
    if style['width'] == 'auto':
        style['width'] = 3 if timeframe == '15m' else 2
    
    if style['dash'] == 'auto':
        style['dash'] = 'solid' if timeframe in ['15m', '1h'] else 'dash'
    
    return style

print("Setup, imports, and configuration loaded!")
print("CONFIGURABLE STYLING: Each indicator can have custom width, opacity, dash style!")

Setup, imports, and configuration loaded!
CONFIGURABLE STYLING: Each indicator can have custom width, opacity, dash style!


In [2]:
# =============================================================================
# LOAD DATA
# =============================================================================

# Load Data with Technical Indicators
from src.indicator_utils import add_indicators


print("Loading Market Data...")
loader = DataLoader()

dfs = loader.load_data_with_indicators(
    symbol=symbol, 
    timeframes=['15m', '1h', 'D', 'W', 'M'],
)

df = dfs['15m'].copy()




# Prepare for visualization (limit to last 1000 candles)
viz_df = df.copy()
max_candles = 5000
if len(viz_df) > max_candles:
    viz_df = viz_df.tail(max_candles)

print(f"Data loaded: {len(viz_df):,} candles ready for {symbol} visualization")

Loading Market Data...
📥 Loading data for BTCUSDT...
 - Adding indicators...
 - Dropping NaNs for...
Data loading complete.
Data loaded: 5,000 candles ready for BTCUSDT visualization


In [3]:
from src.data_processing.add_features import get_vp_levels


def get_volume_profiles(df) -> tuple[dict, dict, dict]:
    def safe_vp(df_slice):
        vp = get_vp_levels(df_slice['close'], df_slice['volume'])
        return {k: 0.0 if np.isnan(v) else v for k, v in vp.items()}

    # Use the last date in the DataFrame, floored to day
    date = df['date'].iloc[-1].floor('D')

    # Daily VP: current day 00:00 to 23:59
    day_data = df[df['date'].dt.floor('D') == date]
    daily_vp = safe_vp(day_data)

    # Previous day VP: previous day 00:00 to 23:59
    prev_date = date - pd.Timedelta(days=1)
    prev_data = df[df['date'].dt.floor('D') == prev_date]
    prev_day_vp = safe_vp(prev_data) if len(prev_data) > 0 else daily_vp

    # Previous week VP: previous full week (Monday 00:00 to Sunday 23:59)
    # Find the Monday of the current week
    weekday = date.weekday()  # Monday=0
    this_monday = date - pd.Timedelta(days=weekday)
    prev_monday = this_monday - pd.Timedelta(days=7)

    week_mask = (df['date'] >= prev_monday) & (df['date'] < this_monday)
    week_data = df[week_mask]
    weekly_vp = safe_vp(week_data) if len(week_data) > 0 else daily_vp

    return daily_vp, prev_day_vp, weekly_vp


vps = get_volume_profiles(viz_df)

In [4]:
# Print the computed volume profile levels
print("Daily VP:", vps[0])
print("Previous Day VP:", vps[1])
print("Weekly VP:", vps[2])

Daily VP: {'poc': np.float64(114533.42569999999), 'vah': np.float64(115423.52140000001), 'val': np.float64(114057.32800000001)}
Previous Day VP: {'poc': np.float64(112343.31150000001), 'vah': np.float64(115389.99), 'val': np.float64(109871.47799999999)}
Weekly VP: {'poc': np.float64(112123.69889999999), 'vah': np.float64(124604.8528), 'val': np.float64(108432.09)}


In [ ]:
# =============================================================================
# CREATE CHART
# =============================================================================

print("Creating Multi-Timeframe Interactive Chart...")

# Create subplot structure
fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02,
    subplot_titles=(f'{symbol} Price with Multi-Timeframe Indicators', 'RSI (Multi-Timeframe)', 'MACD', 'Volume'),
    row_heights=[0.5, 0.2, 0.2, 0.1]
)

# === 1. CANDLESTICK CHART ===
fig.add_trace(go.Candlestick(
    x=viz_df.index, open=viz_df['open'], high=viz_df['high'], low=viz_df['low'], close=viz_df['close'],
    name='OHLC', increasing_line_color='#00ff88', decreasing_line_color='#ff4444',
    increasing_fillcolor='#00ff88', decreasing_fillcolor='#ff4444', line=dict(width=1), opacity=0.8
), row=1, col=1)

# === Add Volume Profile Labels ===
daily_vp, prev_day_vp, weekly_vp = vps

# Add horizontal lines and annotations for daily, previous day, and weekly VP levels
vp_colors = {'poc': '#FFD700', 'vah': '#00FF00', 'val': '#FF00FF'}
vp_labels = {'poc': 'POC', 'vah': 'VAH', 'val': 'VAL'}

for key in ['poc', 'vah', 'val']:
    # Daily
    fig.add_hline(y=daily_vp[key], line_dash="solid", line_color=vp_colors[key], opacity=0.7, row=1, col=1)
    fig.add_annotation(x=viz_df.index[-1], y=daily_vp[key],
                      text=f"Daily {vp_labels[key]}",
                      showarrow=False, yshift=0, xanchor='left', font=dict(color=vp_colors[key], size=10),
                      row=1, col=1)
    # Previous Day
    fig.add_hline(y=prev_day_vp[key], line_dash="dot", line_color=vp_colors[key], opacity=0.5, row=1, col=1)
    fig.add_annotation(x=viz_df.index[-1], y=prev_day_vp[key],
                      text=f"Prev {vp_labels[key]}",
                      showarrow=False, yshift=0, xanchor='left', font=dict(color=vp_colors[key], size=10),
                      row=1, col=1)
    # Weekly
    fig.add_hline(y=weekly_vp[key], line_dash="dash", line_color=vp_colors[key], opacity=0.4, row=1, col=1)
    fig.add_annotation(x=viz_df.index[-1], y=weekly_vp[key],
                      text=f"Week {vp_labels[key]}",
                      showarrow=False, yshift=0, xanchor='left', font=dict(color=vp_colors[key], size=10),
                      row=1, col=1)

# === 2. MAIN CHART INDICATORS ===
for indicator, timeframes in CHART_FEATURES_CONFIG.get('main_chart', {}).items():
    available = get_available_features_by_timeframe(viz_df, indicator, timeframes)
    for tf, col_name in available.items():
        color = TIMEFRAME_COLORS.get(tf, '#ffffff')
        style = get_indicator_style(indicator, tf)
        
        # Prepare trace parameters
        trace_params = {
            'x': viz_df.index,
            'y': viz_df[col_name],
            'mode': 'lines',
            'name': f'{indicator.upper()} ({tf})',
            'line': dict(color=color, width=style['width'], dash=style['dash']),
            'opacity': style['opacity']
        }
        
        # Add fill for Bollinger Bands lower
        if indicator == 'bb_lower' and 'fill' in style:
            trace_params['fill'] = style['fill']
            # Convert hex color to rgba for fill
            hex_color = color.lstrip('#')
            rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
            trace_params['fillcolor'] = f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {style["fill_opacity"]})'
        
        # Generic trace - works for ALL indicators
        fig.add_trace(go.Scatter(**trace_params), row=1, col=1)

# === 3. RSI SUBPLOT ===
for indicator, timeframes in CHART_FEATURES_CONFIG.get('rsi_subplot', {}).items():
    available = get_available_features_by_timeframe(viz_df, indicator, timeframes)
    for tf, col_name in available.items():
        color = TIMEFRAME_COLORS.get(tf, '#ffaa00')
        style = get_indicator_style(indicator, tf)
        
        fig.add_trace(go.Scatter(
            x=viz_df.index, y=viz_df[col_name], mode='lines', name=f'RSI ({tf})',
            line=dict(color=color, width=style['width'], dash=style['dash']),
            opacity=style['opacity']
        ), row=2, col=1)

fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=2, col=1)
fig.add_hline(y=50, line_dash="dot", line_color="gray", opacity=0.3, row=2, col=1)
fig.update_yaxes(title_text="RSI", range=[0, 100], row=2, col=1)

# === 4. MACD SUBPLOT ===
macd_col = signal_col = None
for indicator, timeframes in CHART_FEATURES_CONFIG.get('macd_subplot', {}).items():
    available = get_available_features_by_timeframe(viz_df, indicator, timeframes)
    for tf, col_name in available.items():
        color = TIMEFRAME_COLORS.get(tf, '#ffffff')
        style = get_indicator_style(indicator, tf)
        
        if indicator == 'macd':
            macd_col = col_name
            fig.add_trace(go.Scatter(
                x=viz_df.index, y=viz_df[col_name], mode='lines', name=f'MACD ({tf})',
                line=dict(color='#00ff00', width=style['width'], dash=style['dash']),
                opacity=style['opacity']
            ), row=3, col=1)
        elif indicator == 'macd_signal':
            signal_col = col_name
            fig.add_trace(go.Scatter(
                x=viz_df.index, y=viz_df[col_name], mode='lines', name=f'MACD Signal ({tf})',
                line=dict(color='#ff4444', width=style['width'], dash=style['dash']),
                opacity=style['opacity']
            ), row=3, col=1)

if macd_col and signal_col:
    macd_histogram = viz_df[macd_col] - viz_df[signal_col]
    colors = ['green' if x >= 0 else 'red' for x in macd_histogram]
    fig.add_trace(go.Bar(x=viz_df.index, y=macd_histogram, name='MACD Histogram', 
                        marker_color=colors, opacity=0.5), row=3, col=1)

fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5, row=3, col=1)
fig.update_yaxes(title_text="MACD", row=3, col=1)

# === 5. VOLUME BARS ===
volume_colors = ['#00ff88' if viz_df['close'].iloc[i] >= viz_df['open'].iloc[i] else '#ff4444' for i in range(len(viz_df))]
fig.add_trace(go.Bar(x=viz_df.index, y=viz_df['volume'], name='Volume', marker_color=volume_colors, opacity=0.7), row=4, col=1)

for indicator, timeframes in CHART_FEATURES_CONFIG.get('volume_subplot', {}).items():
    if indicator == 'volume_ma20':
        available = get_available_features_by_timeframe(viz_df, indicator, timeframes)
        for tf, col_name in available.items():
            style = get_indicator_style(indicator, tf)
            fig.add_trace(go.Scatter(
                x=viz_df.index, y=viz_df[col_name], mode='lines', name=f'Volume MA20 ({tf})',
                line=dict(color='#ffff00', width=style['width'], dash=style['dash']),
                opacity=style['opacity']
            ), row=4, col=1)

fig.update_yaxes(title_text="Volume", row=4, col=1)

# === 6. LAYOUT & DISPLAY ===
fig.update_layout(
    title={'text': f'{symbol} - {time_frame} - Multi-Timeframe Technical Analysis<br><sup>Configurable Features | {len(viz_df):,} Candles</sup>',
           'x': 0.5, 'xanchor': 'center', 'font': {'size': 16}},
    height=1200, showlegend=True, template='plotly_dark', font=dict(size=10),
    margin=dict(l=60, r=60, t=100, b=60), plot_bgcolor='#0e1117', paper_bgcolor='#0e1117',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, font=dict(size=9)),
    xaxis=dict(rangeselector=dict(buttons=[
        dict(count=1, label="1D", step="day", stepmode="backward"),
        dict(count=7, label="7D", step="day", stepmode="backward"),
        dict(count=30, label="30D", step="day", stepmode="backward"),
        dict(count=90, label="3M", step="day", stepmode="backward"),
        dict(step="all", label="All")
    ], bgcolor='#262730', bordercolor='#4a4a4a', borderwidth=1), rangeslider=dict(visible=False), type="date"),
    hovermode='x unified', hoverlabel=dict(bgcolor="rgba(0,0,0,0.8)", bordercolor="white", font_size=10)
)

fig.update_yaxes(title_text="Price ($)", row=1, col=1, side='right')
for row in range(1, 5):
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#333333', row=row, col=1)

fig.show()

print("Multi-Timeframe Interactive Chart Displayed!")
print("\nTimeframe Colors:")
for tf, color in TIMEFRAME_COLORS.items():
    print(f"  {tf:4} -> {color}")
print("\nConfigurable Styles:")
for indicator, style in INDICATOR_STYLES.items():
    if indicator != 'default':
        print(f"  {indicator:10} -> opacity={style['opacity']}, width={style['width']}, dash={style['dash']}")


Creating Multi-Timeframe Interactive Chart...
  Missing: rsi_1h


Multi-Timeframe Interactive Chart Displayed!

Timeframe Colors:
  15m  -> #ffd700
  1h   -> #ff6600
  D    -> #ff0066
  W    -> #9370db
  M    -> #00ffff

Configurable Styles:
  bb_upper   -> opacity=0.6, width=1, dash=auto
  bb_lower   -> opacity=0.6, width=1, dash=auto
  vwap       -> opacity=0.9, width=2, dash=dot
  ema5       -> opacity=0.8, width=auto, dash=auto
  ema9       -> opacity=0.8, width=auto, dash=auto
  ema13      -> opacity=0.8, width=auto, dash=auto
  ema20      -> opacity=0.8, width=auto, dash=auto
  ema21      -> opacity=0.8, width=auto, dash=auto
  ema50      -> opacity=0.8, width=auto, dash=auto
  ema200     -> opacity=0.8, width=auto, dash=auto


: 